In [1]:
# This is used to increase the notebook's width to fill the screen, allowing for better plot visualization
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import os
import cv2
import time
import shutil
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

from tqdm import tqdm
from utils import plot
from IPython.display import display, HTML

C:\Users\leoar\AppData\Local\Temp\ipykernel_17920\1903726667.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Loading Dataframes

In [2]:
# Relative path to dataset
data_dir = os.path.join( "C:\\Datasets", "COVID19", "Tomografia", "COVIDx CT-3A" )
assert os.path.exists( data_dir ), "Unable to find the relative path to COVIDx CT-3A, please check data_dir..."

## Samples Dataframe

In [3]:
# Path to metadata csv
csv_path = os.path.join( "..", "metadata", "new_split_metadata.csv" )

# Reads metadata as dataframe, "age" column is treated as str since "N/A" can't be int
samples_df = pd.read_csv(csv_path, sep = ";", na_filter = False, dtype={"age": str})

samples_df.head()

,filename,patient_id,source,class,country,sex,age,partition,slice_selection,x_min,y_min,x_max,y_max,verified_finding,view,modality
0,NCP_96_1328_0032.png,NCP_96,CNCB,COVID-19,China,M,74.0,train,Expert,9,94,512,405,Yes,Axial,CT
1,NCP_96_1328_0035.png,NCP_96,CNCB,COVID-19,China,M,74.0,train,Expert,10,106,512,405,Yes,Axial,CT
2,NCP_96_1328_0036.png,NCP_96,CNCB,COVID-19,China,M,74.0,train,Expert,10,105,512,406,Yes,Axial,CT
3,NCP_96_1328_0037.png,NCP_96,CNCB,COVID-19,China,M,74.0,train,Expert,11,104,512,406,Yes,Axial,CT
4,NCP_96_1328_0038.png,NCP_96,CNCB,COVID-19,China,M,74.0,train,Expert,11,103,512,406,Yes,Axial,CT


## Patients Dataframe

In [4]:
# Creates a new dataframe with metadata sorted by patient
patient_df = plot.convert_df_sample2patient( samples_df )

patient_df.head()

,patient_id,source,class,country,sex,age,partition,slice_selection,verified_finding,view,modality,sample_count
0,NCP_96,CNCB,COVID-19,China,M,74.0,train,Expert,Yes,Axial,CT,121
1,NCP_270,CNCB,COVID-19,China,N/A,N/A,val,Expert,Yes,Axial,CT,124
2,NCP_322,CNCB,COVID-19,China,N/A,N/A,train,Expert,Yes,Axial,CT,40
3,NCP_1276,CNCB,COVID-19,China,N/A,N/A,test,Expert,Yes,Axial,CT,32
4,NCP_2683,CNCB,COVID-19,China,N/A,N/A,test,Expert,Yes,Axial,CT,9


# Stats per Source

In [5]:
def get_sub_dfs(s_df, p_df, src):
    # Extracts the patients dataframe for the current data source
    sub_p_df = p_df[p_df["source"] == src]
    sub_p_df.reset_index(inplace = True)
    
    # Extracts the samples dataframe for the current data source
    sub_s_df = s_df[s_df["source"] == src]
    sub_s_df.reset_index(inplace = True)
    
    return sub_s_df, sub_p_df

def path_from_fname(import_dir, filename):
    return os.path.join(import_dir, "3A_images", filename)

def get_sample_stats(s_df, image_dir):
    
    cols = []
    # cols.extend(s_df.columns)
    new_cols = ["image_height", "image_width", "image_roi_frac", "partition",
                "image_min_pixel_value", "image_avg_pixel_value", "image_max_pixel_value"]
    cols.extend(new_cols)

    stats_dict = { col: [] for col in cols }

    # Converts to dict for faster iteration through rows
    s_dict = s_df.to_dict( "records" )
    for row in tqdm(s_dict):

        # Gets the path for the current file
        path = path_from_fname(image_dir, row["filename"])

        # Reads image if the path exists
        if not os.path.exists(path):
            print(f"\nPath '{path}' does not exist...\n")
            continue
        img = cv2.imread(path, 0)

        # Gets image shape
        H, W = img.shape[:2]

        # Image total area in pixels
        img_area = H * W

        # Area for the inner crop
        roi_h = row["y_max"] - row["y_min"]
        roi_w = row["x_max"] - row["x_min"]
        img_roi_area = roi_h * roi_w

        # Fraction of the image occupied by the inner crop
        img_roi_frac = img_roi_area / img_area

        stats_dict["image_height"].append(H)
        stats_dict["image_width"].append(W)
        stats_dict["image_roi_frac"].append(img_roi_frac)
        stats_dict["partition"].append(row["partition"])
        stats_dict["image_min_pixel_value"].append( np.min(img).astype(int) )
        stats_dict["image_avg_pixel_value"].append( np.mean(img).astype(float) )
        stats_dict["image_max_pixel_value"].append( np.max(img).astype(int) )
        
#         for col in s_df.columns:
#             stats_dict[col].append(row[col])

    sub_sample_stats_df = pd.DataFrame.from_dict(stats_dict)
    return sub_sample_stats_df

def get_interval(values):
    vmin, vmax = np.min(values), np.max(values)
    is_equal = (vmin == vmax)
    if isinstance(vmin, float) or isinstance(vmax, float):
        vmin, vmax = f"{vmin:.3f}", f"{vmax:.3f}"
    if is_equal:
        return f"[{vmin}]"
    return f"[{vmin}, {vmax}]"

def get_distribution(values):
    vavg, vstd = np.mean(values), np.std(values)
    
    if vavg.is_integer():
        vavg = int(vavg)
    
    if vstd.is_integer():
        vstd = int(vstd)
    
    if isinstance(vavg, float) or isinstance(vstd, float):
        vavg, vstd = f"{vavg:.3f}", f"{vstd:.3f}"
    return f"{vavg} +- {vstd}"

def mount_sample_stats_table(s_stats_df):
    table_as_dict = { "Attribute": [], "Interval": [], "Distribution": [] }
    
    for col in s_stats_df.columns:
        table_as_dict["Attribute"].append(col)
        values = s_stats_df[col].to_list()
        
        # Gets the interval for values of the current column
        interval = get_interval(values)
        table_as_dict["Interval"].append(interval)
        
        # Gets the distribution for values of the current column
        distrib = get_distribution(values)
        table_as_dict["Distribution"].append(distrib)

    sample_stats_table = pd.DataFrame.from_dict(table_as_dict)
    sample_stats_table.set_index("Attribute", inplace=True)
    return sample_stats_table

def mount_stats_table_per_partition(s_stats_df):
    stats_table_list = []
    partitions = ["train", "val", "test"]
    
    # "dataset" stands for the whole dataset
    for part in ["dataset", "train", "val", "test"]:
        
        # Filters the input df for the rows corresponding to the current partition
        sub_stats_df = s_stats_df.copy(deep = True)
        if part in partitions:
            sub_stats_df = sub_stats_df[sub_stats_df["partition"] == part]
            sub_stats_df.reset_index(inplace=True, drop=True)
        
        if len(sub_stats_df) == 0:
            continue
            
        # Drops the partition column
        sub_stats_df.drop(columns=["partition"], inplace=True)
        
        part_stats_table = mount_sample_stats_table(sub_stats_df)
        main_header = f"{part.title()} Partition" if part in partitions else part.title()
        header = [ [main_header for _ in part_stats_table.columns], part_stats_table.columns ]
        part_stats_table.columns = header
        
        stats_table_list.append(part_stats_table)
    
    dst_df = pd.concat(stats_table_list, axis=1)
    return dst_df

## radiopaedia.org

In [6]:
src = "radiopaedia.org"
sub_samples_df, sub_patient_df = get_sub_dfs(samples_df, patient_df, src)
print(f"Found {len(sub_samples_df)} samples from {len(sub_patient_df)} patients for source '{src}'")

Found 3574 samples from 105 patients for source 'radiopaedia.org'


### Sample Stats

In [7]:
sub_sample_stats_df = get_sample_stats(sub_samples_df, data_dir)
sample_stats_table = mount_stats_table_per_partition(sub_sample_stats_df)
display(sample_stats_table)

100%|█████████████████████████████████████████████████████████████████████████████| 3574/3574 [00:23<00:00, 151.07it/s]


Dataset                     \
                                Interval       Distribution   
Attribute                                                     
image_height                  [330, 630]  619.400 +- 41.344   
image_width                   [533, 630]  626.617 +- 14.354   
image_roi_frac            [0.340, 1.000]     0.675 +- 0.131   
image_min_pixel_value            [0, 56]    8.675 +- 12.807   
image_avg_pixel_value  [58.607, 219.672]  123.052 +- 19.719   
image_max_pixel_value              [255]           255 +- 0   

                          Test Partition                     
                                Interval       Distribution  
Attribute                                                    
image_height                  [330, 630]  619.400 +- 41.344  
image_width                   [533, 630]  626.617 +- 14.354  
image_roi_frac            [0.340, 1.000]     0.675 +- 0.131  
image_min_pixel_value            [0, 56]    8.675 +- 12.807  
image_avg_pixel_value  [58.607, 219.672]  123.052 +- 19.719  
image_max_pixel_value              [255]           255 +- 0

### Patient Metadata

## COVID-CTset

In [8]:
src = "COVID-CTset"
sub_samples_df, sub_patient_df = get_sub_dfs(samples_df, patient_df, src)
print(f"Found {len(sub_samples_df)} samples from {len(sub_patient_df)} patients for source '{src}'")

Found 12058 samples from 377 patients for source 'COVID-CTset'


### Sample Stats

In [9]:
sub_sample_stats_df = get_sample_stats(sub_samples_df, data_dir)
sample_stats_table = mount_stats_table_per_partition(sub_sample_stats_df)
display(sample_stats_table)

100%|███████████████████████████████████████████████████████████████████████████| 12058/12058 [01:10<00:00, 172.09it/s]


Dataset                     \
                                Interval       Distribution   
Attribute                                                     
image_height                       [512]           512 +- 0   
image_width                        [512]           512 +- 0   
image_roi_frac            [0.324, 0.969]     0.655 +- 0.103   
image_min_pixel_value               [55]            55 +- 0   
image_avg_pixel_value  [94.800, 186.098]  133.354 +- 14.358   
image_max_pixel_value              [255]           255 +- 0   

                         Train Partition                     \
                                Interval       Distribution   
Attribute                                                     
image_height                       [512]           512 +- 0   
image_width                        [512]           512 +- 0   
image_roi_frac            [0.369, 0.922]     0.653 +- 0.102   
image_min_pixel_value               [55]            55 +- 0   
image_avg_pixel_value  [94.800, 182.575]  132.870 +- 14.391   
image_max_pixel_value              [255]           255 +- 0   

                           Val Partition                     \
                                Interval       Distribution   
Attribute                                                     
image_height                       [512]           512 +- 0   
image_width                        [512]           512 +- 0   
image_roi_frac            [0.324, 0.969]     0.657 +- 0.115   
image_min_pixel_value               [55]            55 +- 0   
image_avg_pixel_value  [95.044, 186.098]  134.473 +- 16.298   
image_max_pixel_value              [255]           255 +- 0   

                           Test Partition                     
                                 Interval       Distribution  
Attribute                                                     
image_height                        [512]           512 +- 0  
image_width                         [512]           512 +- 0  
image_roi_frac             [0.382, 0.910]     0.655 +- 0.091  
image_min_pixel_value                [55]            55 +- 0  
image_avg_pixel_value  [103.086, 172.709]  133.684 +- 11.905  
image_max_pixel_value               [255]           255 +- 0

### Patient Metadata

## LIDC-IDRI

In [10]:
src = "LIDC-IDRI"
sub_samples_df, sub_patient_df = get_sub_dfs(samples_df, patient_df, src)
print(f"Found {len(sub_samples_df)} samples from {len(sub_patient_df)} patients for source '{src}'")

Found 3999 samples from 39 patients for source 'LIDC-IDRI'


### Sample Stats

In [11]:
sub_sample_stats_df = get_sample_stats(sub_samples_df, data_dir)
sample_stats_table = mount_stats_table_per_partition(sub_sample_stats_df)
display(sample_stats_table)

100%|█████████████████████████████████████████████████████████████████████████████| 3999/3999 [00:28<00:00, 141.35it/s]


Dataset                     \
                                Interval       Distribution   
Attribute                                                     
image_height                       [512]           512 +- 0   
image_width                        [512]           512 +- 0   
image_roi_frac            [0.372, 0.855]     0.604 +- 0.101   
image_min_pixel_value                [0]             0 +- 0   
image_avg_pixel_value  [89.507, 159.623]  116.165 +- 13.352   
image_max_pixel_value              [255]           255 +- 0   

                         Train Partition                     \
                                Interval       Distribution   
Attribute                                                     
image_height                       [512]           512 +- 0   
image_width                        [512]           512 +- 0   
image_roi_frac            [0.372, 0.855]     0.586 +- 0.106   
image_min_pixel_value                [0]             0 +- 0   
image_avg_pixel_value  [89.507, 156.058]  113.415 +- 13.662   
image_max_pixel_value              [255]           255 +- 0   

                            Val Partition                    \
                                 Interval      Distribution   
Attribute                                                     
image_height                        [512]          512 +- 0   
image_width                         [512]          512 +- 0   
image_roi_frac             [0.419, 0.757]    0.610 +- 0.073   
image_min_pixel_value                 [0]            0 +- 0   
image_avg_pixel_value  [101.500, 148.890]  118.347 +- 9.745   
image_max_pixel_value               [255]          255 +- 0   

                          Test Partition                     
                                Interval       Distribution  
Attribute                                                    
image_height                       [512]           512 +- 0  
image_width                        [512]           512 +- 0  
image_roi_frac            [0.400, 0.825]     0.647 +- 0.095  
image_min_pixel_value                [0]             0 +- 0  
image_avg_pixel_value  [97.349, 159.623]  121.926 +- 13.141  
image_max_pixel_value              [255]           255 +- 0

### Patient Metadata

## Stony Brook

In [12]:
src = "Stony Brook"
sub_samples_df, sub_patient_df = get_sub_dfs(samples_df, patient_df, src)
print(f"Found {len(sub_samples_df)} samples from {len(sub_patient_df)} patients for source '{src}'")

Found 14461 samples from 90 patients for source 'Stony Brook'


### Sample Stats

In [13]:
sub_sample_stats_df = get_sample_stats(sub_samples_df, data_dir)
sample_stats_table = mount_stats_table_per_partition(sub_sample_stats_df)
display(sample_stats_table)

100%|███████████████████████████████████████████████████████████████████████████| 14461/14461 [01:43<00:00, 140.38it/s]


Dataset                     \
                                Interval       Distribution   
Attribute                                                     
image_height                       [512]           512 +- 0   
image_width                        [512]           512 +- 0   
image_roi_frac            [0.207, 0.914]     0.625 +- 0.096   
image_min_pixel_value            [0, 55]     0.654 +- 5.963   
image_avg_pixel_value  [87.306, 169.083]  123.895 +- 12.346   
image_max_pixel_value              [255]           255 +- 0   

                         Train Partition                     \
                                Interval       Distribution   
Attribute                                                     
image_height                       [512]           512 +- 0   
image_width                        [512]           512 +- 0   
image_roi_frac            [0.318, 0.914]     0.621 +- 0.091   
image_min_pixel_value                [0]             0 +- 0   
image_avg_pixel_value  [92.298, 161.767]  121.977 +- 10.661   
image_max_pixel_value              [255]           255 +- 0   

                           Val Partition                     \
                                Interval       Distribution   
Attribute                                                     
image_height                       [512]           512 +- 0   
image_width                        [512]           512 +- 0   
image_roi_frac            [0.207, 0.795]     0.592 +- 0.095   
image_min_pixel_value            [0, 55]    3.285 +- 13.033   
image_avg_pixel_value  [87.306, 169.083]  121.019 +- 13.504   
image_max_pixel_value              [255]           255 +- 0   

                          Test Partition                     
                                Interval       Distribution  
Attribute                                                    
image_height                       [512]           512 +- 0  
image_width                        [512]           512 +- 0  
image_roi_frac            [0.337, 0.883]     0.671 +- 0.093  
image_min_pixel_value                [0]             0 +- 0  
image_avg_pixel_value  [98.438, 157.284]  132.531 +- 12.052  
image_max_pixel_value              [255]           255 +- 0

### Patient Metadata

## COVID-CT-MD

In [14]:
src = "COVID-CT-MD"
sub_samples_df, sub_patient_df = get_sub_dfs(samples_df, patient_df, src)
print(f"Found {len(sub_samples_df)} samples from {len(sub_patient_df)} patients for source '{src}'")

Found 23280 samples from 301 patients for source 'COVID-CT-MD'


### Sample Stats

In [15]:
sub_sample_stats_df = get_sample_stats(sub_samples_df, data_dir)
sample_stats_table = mount_stats_table_per_partition(sub_sample_stats_df)
display(sample_stats_table)

100%|███████████████████████████████████████████████████████████████████████████| 23280/23280 [02:52<00:00, 135.16it/s]


Dataset                     \
                                Interval       Distribution   
Attribute                                                     
image_height                       [512]           512 +- 0   
image_width                        [512]           512 +- 0   
image_roi_frac            [0.316, 1.000]     0.675 +- 0.099   
image_min_pixel_value               [55]            55 +- 0   
image_avg_pixel_value  [93.627, 204.452]  139.933 +- 15.752   
image_max_pixel_value              [255]           255 +- 0   

                          Train Partition                     \
                                 Interval       Distribution   
Attribute                                                      
image_height                        [512]           512 +- 0   
image_width                         [512]           512 +- 0   
image_roi_frac             [0.336, 1.000]     0.672 +- 0.099   
image_min_pixel_value                [55]            55 +- 0   
image_avg_pixel_value  [104.357, 197.704]  139.144 +- 15.991   
image_max_pixel_value               [255]           255 +- 0   

                            Val Partition                     \
                                 Interval       Distribution   
Attribute                                                      
image_height                        [512]           512 +- 0   
image_width                         [512]           512 +- 0   
image_roi_frac             [0.418, 0.936]     0.678 +- 0.086   
image_min_pixel_value                [55]            55 +- 0   
image_avg_pixel_value  [108.186, 204.452]  140.549 +- 13.724   
image_max_pixel_value               [255]           255 +- 0   

                          Test Partition                     
                                Interval       Distribution  
Attribute                                                    
image_height                       [512]           512 +- 0  
image_width                        [512]           512 +- 0  
image_roi_frac            [0.316, 0.975]     0.680 +- 0.107  
image_min_pixel_value               [55]            55 +- 0  
image_avg_pixel_value  [93.627, 201.093]  141.771 +- 16.678  
image_max_pixel_value              [255]           255 +- 0

### Patient Metadata

## iCTCF

In [16]:
src = "iCTCF"
sub_samples_df, sub_patient_df = get_sub_dfs(samples_df, patient_df, src)
print(f"Found {len(sub_samples_df)} samples from {len(sub_patient_df)} patients for source '{src}'")

Found 45912 samples from 709 patients for source 'iCTCF'


### Sample Stats

In [17]:
sub_sample_stats_df = get_sample_stats(sub_samples_df, data_dir)
sample_stats_table = mount_stats_table_per_partition(sub_sample_stats_df)
display(sample_stats_table)

100%|███████████████████████████████████████████████████████████████████████████| 45912/45912 [05:46<00:00, 132.50it/s]


Dataset                     \
                                Interval       Distribution   
Attribute                                                     
image_height                       [512]           512 +- 0   
image_width                        [512]           512 +- 0   
image_roi_frac            [0.136, 0.928]     0.546 +- 0.102   
image_min_pixel_value            [0, 59]   39.195 +- 13.548   
image_avg_pixel_value  [30.096, 166.213]  109.593 +- 16.087   
image_max_pixel_value              [255]           255 +- 0   

                         Train Partition                     \
                                Interval       Distribution   
Attribute                                                     
image_height                       [512]           512 +- 0   
image_width                        [512]           512 +- 0   
image_roi_frac            [0.217, 0.928]     0.545 +- 0.101   
image_min_pixel_value            [0, 59]   39.772 +- 12.262   
image_avg_pixel_value  [41.338, 166.213]  109.592 +- 16.130   
image_max_pixel_value              [255]           255 +- 0   

                           Val Partition                     \
                                Interval       Distribution   
Attribute                                                     
image_height                       [512]           512 +- 0   
image_width                        [512]           512 +- 0   
image_roi_frac            [0.244, 0.787]     0.538 +- 0.104   
image_min_pixel_value            [0, 59]   36.692 +- 14.885   
image_avg_pixel_value  [69.869, 164.416]  107.246 +- 15.870   
image_max_pixel_value              [255]           255 +- 0   

                          Test Partition                     
                                Interval       Distribution  
Attribute                                                    
image_height                       [512]           512 +- 0  
image_width                        [512]           512 +- 0  
image_roi_frac            [0.136, 0.916]     0.558 +- 0.100  
image_min_pixel_value            [0, 59]   39.855 +- 15.575  
image_avg_pixel_value  [30.096, 158.079]  111.914 +- 15.828  
image_max_pixel_value              [255]           255 +- 0

### Patient Metadata

## CNCB

In [18]:
src = "CNCB"
sub_samples_df, sub_patient_df = get_sub_dfs(samples_df, patient_df, src)
print(f"Found {len(sub_samples_df)} samples from {len(sub_patient_df)} patients for source '{src}'")

Found 115837 samples from 1854 patients for source 'CNCB'


### Sample Stats

In [19]:
sub_sample_stats_df = get_sample_stats(sub_samples_df, data_dir)
sample_stats_table = mount_stats_table_per_partition(sub_sample_stats_df)
display(sample_stats_table)

100%|█████████████████████████████████████████████████████████████████████████| 115837/115837 [14:17<00:00, 135.13it/s]


Dataset                     \
                                Interval       Distribution   
Attribute                                                     
image_height                 [256, 1168]  513.156 +- 22.998   
image_width                  [256, 1168]  513.156 +- 22.998   
image_roi_frac            [0.080, 0.869]     0.545 +- 0.088   
image_min_pixel_value            [0, 72]   30.232 +- 17.008   
image_avg_pixel_value  [41.923, 198.523]  122.175 +- 19.236   
image_max_pixel_value         [251, 255]   255.000 +- 0.012   

                         Train Partition                     \
                                Interval       Distribution   
Attribute                                                     
image_height                 [256, 1168]  512.913 +- 19.317   
image_width                  [256, 1168]  512.913 +- 19.317   
image_roi_frac            [0.080, 0.846]     0.544 +- 0.088   
image_min_pixel_value            [0, 72]   30.378 +- 16.901   
image_avg_pixel_value  [41.923, 197.095]  122.274 +- 19.552   
image_max_pixel_value         [251, 255]   255.000 +- 0.015   

                           Val Partition                     \
                                Interval       Distribution   
Attribute                                                     
image_height                 [512, 1024]  513.659 +- 29.100   
image_width                  [512, 1024]  513.659 +- 29.100   
image_roi_frac            [0.148, 0.869]     0.542 +- 0.091   
image_min_pixel_value            [0, 64]   30.057 +- 17.120   
image_avg_pixel_value  [52.975, 185.847]  121.360 +- 19.049   
image_max_pixel_value              [255]           255 +- 0   

                          Test Partition                     
                                Interval       Distribution  
Attribute                                                    
image_height                 [512, 1024]  513.400 +- 26.247  
image_width                  [512, 1024]  513.400 +- 26.247  
image_roi_frac            [0.085, 0.816]     0.551 +- 0.086  
image_min_pixel_value            [0, 72]   29.962 +- 17.217  
image_avg_pixel_value  [63.687, 198.523]  122.687 +- 18.405  
image_max_pixel_value              [255]           255 +- 0

### Patient Metadata

## COVID-19-CT-Seg

In [20]:
src = "COVID-19-CT-Seg"
sub_samples_df, sub_patient_df = get_sub_dfs(samples_df, patient_df, src)
print(f"Found {len(sub_samples_df)} samples from {len(sub_patient_df)} patients for source '{src}'")

Found 1726 samples from 18 patients for source 'COVID-19-CT-Seg'


### Sample Stats

In [21]:
sub_sample_stats_df = get_sample_stats(sub_samples_df, data_dir)
sample_stats_table = mount_stats_table_per_partition(sub_sample_stats_df)
display(sample_stats_table)

100%|█████████████████████████████████████████████████████████████████████████████| 1726/1726 [00:12<00:00, 133.17it/s]


Dataset                     \
                                Interval       Distribution   
Attribute                                                     
image_height                  [401, 630]  531.621 +- 58.461   
image_width                   [512, 630]  545.021 +- 52.972   
image_roi_frac            [0.416, 0.927]     0.607 +- 0.093   
image_min_pixel_value            [0, 55]   41.502 +- 22.080   
image_avg_pixel_value  [65.915, 166.446]  128.157 +- 13.183   
image_max_pixel_value              [255]           255 +- 0   

                          Test Partition                     
                                Interval       Distribution  
Attribute                                                    
image_height                  [401, 630]  531.621 +- 58.461  
image_width                   [512, 630]  545.021 +- 52.972  
image_roi_frac            [0.416, 0.927]     0.607 +- 0.093  
image_min_pixel_value            [0, 55]   41.502 +- 22.080  
image_avg_pixel_value  [65.915, 166.446]  128.157 +- 13.183  
image_max_pixel_value              [255]           255 +- 0

### Patient Metadata

## TCIA

In [22]:
src = "TCIA"
sub_samples_df, sub_patient_df = get_sub_dfs(samples_df, patient_df, src)
print(f"Found {len(sub_samples_df)} samples from {len(sub_patient_df)} patients for source '{src}'")

Found 11816 samples from 643 patients for source 'TCIA'


### Sample Stats

In [23]:
sub_sample_stats_df = get_sample_stats(sub_samples_df, data_dir)
sample_stats_table = mount_stats_table_per_partition(sub_sample_stats_df)
display(sample_stats_table)

100%|███████████████████████████████████████████████████████████████████████████| 11816/11816 [01:06<00:00, 177.09it/s]


Dataset                       Test Partition  \
                                Interval      Distribution           Interval   
Attribute                                                                       
image_height                       [512]          512 +- 0              [512]   
image_width                        [512]          512 +- 0              [512]   
image_roi_frac            [0.135, 0.830]    0.444 +- 0.084     [0.135, 0.830]   
image_min_pixel_value            [0, 55]    1.634 +- 8.954            [0, 55]   
image_avg_pixel_value  [67.910, 146.189]  98.787 +- 11.239  [67.910, 146.189]   
image_max_pixel_value              [255]          255 +- 0              [255]   

                                         
                           Distribution  
Attribute                                
image_height                   512 +- 0  
image_width                    512 +- 0  
image_roi_frac           0.444 +- 0.084  
image_min_pixel_value    1.634 +- 8.954  
image_avg_pixel_value  98.787 +- 11.239  
image_max_pixel_value          255 +- 0

### Patient Metadata

## STOIC

In [24]:
src = "STOIC"
sub_samples_df, sub_patient_df = get_sub_dfs(samples_df, patient_df, src)
print(f"Found {len(sub_samples_df)} samples from {len(sub_patient_df)} patients for source '{src}'")

Found 192361 samples from 1176 patients for source 'STOIC'


### Sample Stats

In [25]:
sub_sample_stats_df = get_sample_stats(sub_samples_df, data_dir)
sample_stats_table = mount_stats_table_per_partition(sub_sample_stats_df)
display(sample_stats_table)

100%|█████████████████████████████████████████████████████████████████████████| 192361/192361 [20:34<00:00, 155.80it/s]


Dataset                     \
                                Interval       Distribution   
Attribute                                                     
image_height                       [512]           512 +- 0   
image_width                        [512]           512 +- 0   
image_roi_frac            [0.261, 1.000]     0.621 +- 0.106   
image_min_pixel_value            [0, 57]   21.166 +- 26.762   
image_avg_pixel_value  [70.965, 201.175]  125.139 +- 17.062   
image_max_pixel_value              [255]           255 +- 0   

                          Test Partition                     
                                Interval       Distribution  
Attribute                                                    
image_height                       [512]           512 +- 0  
image_width                        [512]           512 +- 0  
image_roi_frac            [0.261, 1.000]     0.621 +- 0.106  
image_min_pixel_value            [0, 57]   21.166 +- 26.762  
image_avg_pixel_value  [70.965, 201.175]  125.139 +- 17.062  
image_max_pixel_value              [255]           255 +- 0

### Patient Metadata